In [1]:
!pip install -U finance-datareader
!pip install torch torchvision

In [0]:
import FinanceDataReader as fdr
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import time

In [0]:
#이동평균 칼럼 추가, 당일 포함 
def MA(df,day):
    
    df['MA_{}'.format(day)] = df['Close'].rolling(window=day).mean() 
    #moveAverage(data,30) 30일 이동 평균 추가

In [0]:
#지수 이동 평균
def EMA(df, day):
    df['EMA_{}'.format(day)] = df['Close'].ewm(span=day).mean()


In [0]:
# 이격도(PPO)
def PPO(df, day):
    df['PPO_{}'.format(day)] = (df['Close']/df['MA_{}'.format(day)])*100
    

In [0]:
#MACD
def MACD(df):
    
    ma_12=df['Close'].ewm(span=12).mean() # 단기(12)
    ma_26 = df['Close'].ewm(span=26).mean() # 장기(26) EMA
    macd = ma_12 - ma_26 # MACD
    macdSignal = macd.ewm(span=9).mean() # Signal
    macdOscillator = macd - macdSignal # Oscillator
    df['MACD'] = macdOscillator

In [0]:
#CCI
def CCI(df):
    #CCI = (M-N) / (0.015*D)
    # M=특정일의 고가,저가, 종가의 평균
    # N = 일정기간동안의 단순이동평균 통상적으로 20일로 사용
    # D = M-N의 일정기간동안의 단순이동평균
    M = ((df.High)+(df.Low)+(df.Close)) / 3
    N = M.rolling(20).mean()
    D = (M-N).rolling(20).mean()
    CCI = (M - N)/ (0.015 * D)
    df['CCI'] = CCI

In [0]:
#RSI
def U(x):
    if x >= 0:
        return x
    else :
        return 0
    
def D(x):
    if x <= 0:
        return x*(-1)
    else :
        return 0

def RSI(df):
    df['diff_rsi'] = (df['Close'].shift(1)-df['Close'])
    df['AU'] = df['diff_rsi'].apply(U).rolling(window=14).mean() 
    df['AD'] = df['diff_rsi'].apply(D).rolling(window=14).mean() 
    df['RSI'] = df['AU']/(df['AU']+df['AD'])

In [0]:
#모멘텀 스토캐스틱
def MOMENT(df,day):
    temp_df = df.copy()
    temp_df = temp_df.reset_index()
    for i in range(len(temp_df)-day+1):
        temp_df.loc[i, 'high_st']= temp_df[i:i+day]['High'].max()
        temp_df.loc[i, 'low_st']= temp_df[i:i+day]['Low'].min()
        
    temp_df['high_st_4'] = temp_df['high_st'].shift(4)
    temp_df['low_st_4'] = temp_df['low_st'].shift(4)
    
    temp_df['fast_K'] = (temp_df['Close']-temp_df['low_st_4'])/(temp_df['high_st_4']-temp_df['low_st_4'])
    temp_df['fast_D'] = temp_df['fast_K'].rolling(3).mean()
    temp_df['slow_K'] = temp_df['fast_D']
    temp_df['slow_D'] = temp_df['slow_K'].rolling(3).mean()
    temp_df = temp_df.set_index('Date')
    return temp_df

In [0]:
data = fdr.DataReader('068270',"2019-1-1","2020-5-13")


In [0]:
MA(data,30)
EMA(data,30)
PPO(data,30)
MACD(data)
RSI(data)
CCI(data)
m = MOMENT(data,30)

In [24]:
data['MA_30']


Date
2019-01-02              NaN
2019-01-03              NaN
2019-01-04              NaN
2019-01-07              NaN
2019-01-08              NaN
                  ...      
2020-05-07    204133.333333
2020-05-08    205316.666667
2020-05-11    206200.000000
2020-05-12    207133.333333
2020-05-13    208250.000000
Name: MA_30, Length: 335, dtype: float64

In [25]:
data['EMA_30']

Date
2019-01-02    204879.000000
2019-01-03    202904.816667
2019-01-04    205476.206220
2019-01-07    205706.222327
2019-01-08    205300.496074
                  ...      
2020-05-07    202346.780452
2020-05-08    202872.794617
2020-05-11    203364.872383
2020-05-12    203857.461262
2020-05-13    204576.334729
Name: EMA_30, Length: 335, dtype: float64

In [26]:
data['PPO_30']

Date
2019-01-02           NaN
2019-01-03           NaN
2019-01-04           NaN
2019-01-07           NaN
2019-01-08           NaN
                 ...    
2020-05-07     99.934683
2020-05-08    102.524556
2020-05-11    102.085354
2020-05-12    101.866752
2020-05-13    103.241297
Name: PPO_30, Length: 335, dtype: float64

In [29]:
data['MACD']

Date
2019-01-02       0.000000
2019-01-03     -38.101140
2019-01-04     129.704276
2019-01-07      69.310185
2019-01-08     -29.742858
                 ...     
2020-05-07   -2294.429267
2020-05-08   -1942.784424
2020-05-11   -1676.140024
2020-05-12   -1441.021100
2020-05-13   -1011.340856
Name: MACD, Length: 335, dtype: float64

In [36]:
data['CCI']

Date
2019-01-02          NaN
2019-01-03          NaN
2019-01-04          NaN
2019-01-07          NaN
2019-01-08          NaN
                ...    
2020-05-07   -33.908905
2020-05-08    -7.725660
2020-05-11     5.772787
2020-05-12    13.111134
2020-05-13    18.016916
Name: CCI, Length: 335, dtype: float64

In [30]:
data['RSI']

Date
2019-01-02         NaN
2019-01-03         NaN
2019-01-04         NaN
2019-01-07         NaN
2019-01-08         NaN
                ...   
2020-05-07    0.688525
2020-05-08    0.575342
2020-05-11    0.563380
2020-05-12    0.500000
2020-05-13    0.457143
Name: RSI, Length: 335, dtype: float64

In [32]:
m

,Open,High,Low,Close,Volume,Change,MA_30,EMA_30,PPO_30,MACD,diff_rsi,AU,AD,RSI,high_st,low_st,high_st_4,low_st_4,fast_K,fast_D,slow_K,slow_D
Date,,,,,,,,,,,,,,,,,,,,,,
2019-01-02,214430,215862,203445,204879,1045357,-0.035954,NaN,204879.000000,NaN,0.000000,NaN,NaN,NaN,NaN,216817.0,185775.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,204401,207744,199147,201058,951894,-0.018650,NaN,202904.816667,NaN,-38.101140,3821.0,NaN,NaN,NaN,216817.0,185775.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,201058,211087,198670,210132,1132255,0.045131,NaN,205476.206220,NaN,129.704276,-9074.0,NaN,NaN,NaN,216817.0,185775.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-07,216817,216817,205356,206311,1071752,-0.018184,NaN,205706.222327,NaN,69.310185,3821.0,NaN,NaN,NaN,216817.0,185775.0,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-08,206788,209176,202968,203923,646883,-0.011575,NaN,205300.496074,NaN,-29.742858,2388.0,NaN,NaN,NaN,211564.0,185775.0,216817.0,185775.0,0.584627,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-07,206000,207000,203500,204000,402708,-0.014493,204133.333333,202346.780452,99.934683,-2294.429267,3000.0,1500.000000,678.571429,0.688525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-08,206000,214000,205000,210500,895729,0.031863,205316.666667,202872.794617,102.524556,-1942.784424,-6500.0,1500.000000,1107.142857,0.575342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-11,215000,215500,210000,210500,695048,0.000000,206200.000000,203364.872383,102.085354,-1676.140024,0.0,1428.571429,1107.142857,0.563380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
